In [1]:
# Tang Kit Lim (WQD190032)

import datetime
import numpy as np 
import pandas as pd
import pymongo
from pathlib import Path
from geopy.distance import distance

## Load Data From Document Data Store

In [2]:
# MongoDB setting
mongo_host = "mongodb://localhost:27017/"
mongo_client = pymongo.MongoClient(mongo_host)
db = mongo_client.WQD7005

# File Destination to save preprocessed Data
data_directory = '../data/processed/'
Path(data_directory).mkdir(parents=True, exist_ok=True)
township_csv = data_directory + 'edgeprop_townships_preprocessed.csv'
transaction_csv = data_directory + 'edgeprop_transactions_preprocessed.csv'
poi_csv = data_directory + 'iproperty_pois_preprocessed.csv'



In [3]:
# load townships data from mongodb

townships = pd.DataFrame(list(
    db.edgeprop_townships.aggregate([
        {
            '$project': {
                '_id': 0,
                'asset_id': 0,
                'street_name': 0,
                'transacted_price': 0,
                'unit_price_psf': 0,
                'contract_date': 0,
                'propsubtype': 0,
                'tenure': 0,
                'floor': 0,
                'area_sqft': 0,
                'fieldtransactions': 0,
                'non_landed': 0
            }
        }
    ])
))

# Rename columns
townships.columns = [
    'project_id', 'latitude', 'longitude', 'township', 'state', 'area', 'median_psf', 'median_price'
]



In [4]:
# extract pois data from mongoDB

pois = pd.DataFrame(list(
    db.iproperty_pois.aggregate([
        {
            '$addFields': {
                'latitude': '$geometry.location.lat', 
                'longitude': '$geometry.location.lng'
            }
        }, {
            '$project': {
                '_id': 0, 
                'subTypeLabel': 0, 
                'subTypeExtra': 0, 
                'geometry': 0, 
                'placeId': 0, 
                'completionYear': 0, 
                'city': 0, 
                'district': 0, 
                'publicType': 0, 
                'curriculumOffered': 0, 
                '__typename': 0
            }
        }
    ])
))

# Rename columns
pois.columns = [
    'name', 'sub_type', 'category', 'line_name', 'type', 'latitude', 'longitude'
]


In [5]:
# load transactions data from mongodb

transactions = pd.DataFrame(list(
    db.edgeprop_transactions.aggregate([
        {
            '$project': {
                '_id': 0,
                'street_name': 0
            }
        }
    ])
))

# Rename columns
transactions.columns = [
    'project_id', 'project_name', 'transacted_price', 'unit_price_psf', 'date', 'property_type', 'tenure', 'floor', 'area_sqft',
    'non_landed', 'bedrooms', 'psf', 'price', 'state', 'planning_region'
]



## Add new POI related columns to townships dataset


In [6]:
pois['category'].value_counts()

education         4841
healthcare         842
transportation     256
Name: category, dtype: int64

In [7]:
# create new column to pois, coordinates as a tuple of latitue & longitude
pois['coordinates'] = list(zip(round(pois['latitude'], 4), round(pois['longitude'], 4)))
progress_count = 0

# calculate distance between two points
def calc_distance(source, target):
    return(distance(source, target).m)


# find nearby township psf
def nearby_poi(target_coordinate):
    global progress_count
    
    # add new column to record distance of POI to target coordinate
    pois['distance'] = pois['coordinates'].apply(
        lambda x: 
        calc_distance(x, target_coordinate)
    )

    # compute summary of total POI counts within 3km for each category
    # add distance to the nearest POI of selected type for each category
    nearby_pois = (
        pois[(pois['distance'] <= 3000) & (pois['category'] == 'education')].count()['name'],
        pois[(pois['distance'] <= 3000) & (pois['category'] == 'transportation')].count()['name'],
        pois[(pois['distance'] <= 3000) & (pois['category'] == 'healthcare')].count()['name'],
        pois[(pois['category'] == 'education')]['distance'].min(),
        pois[(pois['category'] == 'transportation') & (pd.notna(pois['line_name']))]['distance'].min(),
        pois[(pois['category'] == 'healthcare') & (pois['sub_type'] == 'hospital')]['distance'].min()
    )

    # drop the distance column
    pois.drop(['distance'], axis=1, inplace=True)

    if (progress_count % 10) == 0:
        print('.', end='')
    
    progress_count = progress_count + 1
    
    return (nearby_pois)




In [8]:
# drop rows without gps coordination
townships[['latitude', 'longitude']].replace(['0', 0], np.nan, inplace=True)
townships.dropna(subset = ['latitude', 'longitude'], inplace = True)

/home/kitlim/py3_kernel/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [9]:
# create new column 'coordinates' as a tuple of latitue & longitude to townships
townships['coordinates'] = list(zip(townships['latitude'], townships['longitude']))

# add new column (tuple of information) to Township to record near by POI information
print('Start', datetime.datetime.now())
townships['nearby_pois'] = townships.apply(
    lambda x: nearby_poi((x['coordinates'])), axis=1
)
print()
print('Completed', datetime.datetime.now())

Start 2020-06-19 09:04:55.556443
.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
Completed 2020-06-19 10:49:07.872012


In [10]:
# split the tuple returned by the fuction into three new columns 
# 'nearby_poi_education', 'nearby_poi_transportation', 'nearby_poi_healthcare',
# 'nearest_school', 'nearest_train_station' and 'nearest_hospital'

townships[[
    'poi_nearby_education', 'poi_nearby_transportation', 'poi_nearby_healthcare', 
    'nearest_school', 'nearest_train_station', 'nearest_hospital'
]] = pd.DataFrame(townships['nearby_pois'].tolist(), index=townships.index)                                                                                                                       

townships['total_poi_nearby'] = townships['poi_nearby_education'] + townships['poi_nearby_transportation'] + townships['poi_nearby_healthcare']
townships['nearest_poi'] = townships[['nearest_school', 'nearest_train_station', 'nearest_hospital']].min(axis=1)
townships.drop(['coordinates', 'nearby_pois'], axis=1, inplace=True)
townships


,project_id,latitude,longitude,township,state,area,median_psf,median_price,poi_nearby_education,poi_nearby_transportation,poi_nearby_healthcare,nearest_school,nearest_train_station,nearest_hospital,total_poi_nearby,nearest_poi
1,28054,1.850839,103.098501,Taman Melewar 2,Johor,Parit Raja,33.0,140000,7,0,3,698.877585,118249.246278,16451.842589,10,698.877585
3,19000,2.424192,103.837082,Taman Mersing,Johor,Mersing,113.0,332500,10,0,2,583.035476,179112.693473,1101.439888,12,583.035476
4,19009,2.502480,102.831990,"Taman Mewah, Sungai Segamat",Johor,Segamat,67.0,185000,16,0,4,479.442036,67412.610445,1880.370354,20,479.442036
5,19036,1.989595,102.873294,"Taman Mutiara, Parit Sulong",Johor,Parit Sulong,146.0,329700,7,0,2,642.169632,88940.537416,15543.042648,9,642.169632
9,28046,2.109805,103.396274,Taman Perdana,Johor,Kluang,101.0,213125,4,0,1,377.403414,135795.961947,10312.273554,5,377.403414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7466,23082,3.016821,101.623228,Bandar Puteri Puchong,Selangor,Bandar Puteri Puchong,539.0,1174054,19,4,3,851.781025,1181.540717,777.226157,26,777.226157
7467,11372,3.053950,101.528999,"Seksyen 19, Shah Alam",Selangor,Shah Alam,280.0,260743,21,2,3,330.456470,492.440153,1366.133911,26,330.456470
7468,11479,3.130250,101.473999,Seri Jati & Baiduri,Selangor,Setia Alam/Alam Nusantara,351.0,303397,8,0,0,1549.983563,7098.462905,5893.744567,8,1549.983563
7469,10419,2.876342,101.806980,Bandar Bukit Mahkota Bangi,Selangor,Bangi,249.0,381346,6,0,1,738.335774,3857.170753,8069.968183,7,738.335774


## Re-calculation Values for Certain Columns

In [11]:
# Recalculate / Create new column from other dataset
# 'median_psf' and 'median_price' in townships dataset were calculated based on the last transactions page crawled
# need to recalculate using all transactions we have crawled


def calculate_median_price(project_id):
    return transactions[transactions['project_id'] == project_id]['price'].median()


def calculate_median_psf(project_id):
    return transactions[transactions['project_id'] == project_id]['psf'].median()


def transaction_count(project_id):
    return transactions[transactions['project_id'] == project_id].shape[0]


townships['median_price'] = townships['project_id'].apply(
    lambda x: calculate_median_price(x)
)

townships['median_psf'] = townships['project_id'].apply(
    lambda x: calculate_median_psf(x)
)

townships['transaction_count'] = townships['project_id'].apply(
    lambda x: transaction_count(x)
)


/home/kitlim/py3_kernel/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [12]:
# drop rows without area_sqft and recalculate psf
transactions[['area_sqft']].replace(['0', 0], np.nan, inplace = True)
transactions.dropna(subset = ['area_sqft'], inplace = True)
transactions['psf'] = transactions['price']/transactions['area_sqft']

/home/kitlim/py3_kernel/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


## Categorial Data Correction

In [13]:
townships['state'].unique()

array(['Johor', 'Kuala Lumpur', 'Negeri Sembilan', 'Pahang', 'Penang',
       'Perak', 'Putrajaya', 'Sarawak', 'Selangor', 'Selangor '],
      dtype=object)

In [14]:
townships['state'] = townships['state'].str.replace('Selangor ', 'Selangor')
townships

,project_id,latitude,longitude,township,state,area,median_psf,median_price,poi_nearby_education,poi_nearby_transportation,poi_nearby_healthcare,nearest_school,nearest_train_station,nearest_hospital,total_poi_nearby,nearest_poi,transaction_count
1,28054,1.850839,103.098501,Taman Melewar 2,Johor,Parit Raja,32.5,140000.0,7,0,3,698.877585,118249.246278,16451.842589,10,698.877585,4
3,19000,2.424192,103.837082,Taman Mersing,Johor,Mersing,92.5,290000.0,10,0,2,583.035476,179112.693473,1101.439888,12,583.035476,4
4,19009,2.502480,102.831990,"Taman Mewah, Sungai Segamat",Johor,Segamat,56.0,160000.0,16,0,4,479.442036,67412.610445,1880.370354,20,479.442036,4
5,19036,1.989595,102.873294,"Taman Mutiara, Parit Sulong",Johor,Parit Sulong,122.5,324400.0,7,0,2,642.169632,88940.537416,15543.042648,9,642.169632,4
9,28046,2.109805,103.396274,Taman Perdana,Johor,Kluang,102.0,235000.0,4,0,1,377.403414,135795.961947,10312.273554,5,377.403414,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7466,23082,3.016821,101.623228,Bandar Puteri Puchong,Selangor,Bandar Puteri Puchong,542.0,965000.0,19,4,3,851.781025,1181.540717,777.226157,26,777.226157,193
7467,11372,3.053950,101.528999,"Seksyen 19, Shah Alam",Selangor,Shah Alam,271.0,200000.0,21,2,3,330.456470,492.440153,1366.133911,26,330.456470,193
7468,11479,3.130250,101.473999,Seri Jati & Baiduri,Selangor,Setia Alam/Alam Nusantara,NaN,NaN,8,0,0,1549.983563,7098.462905,5893.744567,8,1549.983563,0
7469,10419,2.876342,101.806980,Bandar Bukit Mahkota Bangi,Selangor,Bangi,246.0,382500.0,6,0,1,738.335774,3857.170753,8069.968183,7,738.335774,188


In [15]:
transactions['tenure'].value_counts()

Freehold     141390
Leasehold     72764
FREEHOLD       3724
Name: tenure, dtype: int64

In [16]:
transactions['tenure'] = transactions['tenure'].str.replace('FREEHOLD', 'Freehold')

In [17]:
transactions['state'].unique()

array(['Selangor', 'Johor', 'Penang', 'Kuala Lumpur', 'selangor'],
      dtype=object)

In [18]:
transactions['state'] = transactions['state'].str.replace('selangor', 'Selangor')

## Drop Missing Categorical Data

In [19]:
townships['area'].isna().sum()

0

In [20]:
townships.dropna(subset = ['area'], inplace = True)

In [21]:
transactions['planning_region'].isna().sum()

64

In [22]:
transactions.dropna(subset = ['planning_region'], inplace = True)

## Fill Missing Numerical Data

In [23]:
transactions[pd.isna(transactions['bedrooms'])]['property_type'].value_counts()

Hotel/Service Apartment    1245
Terrace House                19
Detached House                5
Flat                          2
Condominium/Apartment         1
Semi-Detached House           1
Name: property_type, dtype: int64

In [24]:
# fill na for bedrooms
# for service apartment, fill room with 0
# for other property type, fill room with mode of its type

mode_bedroom={}
for property_type in transactions['property_type'].unique():
    mode_bedroom[property_type] = transactions[transactions['property_type'] == property_type]['bedrooms'].mode()[0]


def fill_bedroom(property_type):
    if property_type == 'Hotel/Service Apartment':
        return 0
    else:
        return mode_bedroom[property_type]
    
    
transactions['bedrooms'] = transactions[['bedrooms', 'property_type']].apply(
    { lambda x: fill_bedroom(x['property_type']) if pd.isna(x['bedrooms']) else x['bedrooms'] }, 
    axis = 1 
)


## Drop Duplicates Observations

In [25]:
pois.drop_duplicates(inplace = True)
pois

,name,sub_type,category,line_name,type,latitude,longitude,coordinates
0,Sekolah Kebangsaan Bedup,primary,education,None,Public School,1.090000,110.640000,"(1.09, 110.64)"
1,Sekolah Kebangsaan Semukoi,primary,education,None,Public School,1.050595,110.674467,"(1.0506, 110.6745)"
2,Sekolah Kebangsaan Entayan,primary,education,None,Public School,1.072190,110.687622,"(1.0722, 110.6876)"
3,Sekolah Kebangsaan Sumpas,primary,education,None,Public School,1.032530,110.647259,"(1.0325, 110.6473)"
4,Sekolah Menengah Kebangsaan Serian,secondary,education,None,Public School,1.170000,110.570000,"(1.17, 110.57)"
...,...,...,...,...,...,...,...,...
5934,Sekolah Menengah Kebangsaan St Peter Kudat,secondary,education,None,Public School,6.895530,116.855420,"(6.8955, 116.8554)"
5935,Sekolah Jenis Kebangsaan (Cina) Lok Yuk Pinang...,primary,education,None,Public School,6.930180,116.817370,"(6.9302, 116.8174)"
5936,Sekolah Menengah Kebangsaan Kudat,secondary,education,None,Public School,6.884280,116.848820,"(6.8843, 116.8488)"
5937,Hospital Kudat,hospital,healthcare,None,Government,6.907740,116.838140,"(6.9077, 116.8381)"


In [26]:
townships.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6762 entries, 1 to 7470
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   project_id                 6762 non-null   int64  
 1   latitude                   6762 non-null   float64
 2   longitude                  6762 non-null   float64
 3   township                   6762 non-null   object 
 4   state                      6762 non-null   object 
 5   area                       6762 non-null   object 
 6   median_psf                 6715 non-null   float64
 7   median_price               6717 non-null   float64
 8   poi_nearby_education       6762 non-null   int64  
 9   poi_nearby_transportation  6762 non-null   int64  
 10  poi_nearby_healthcare      6762 non-null   int64  
 11  nearest_school             6762 non-null   float64
 12  nearest_train_station      6762 non-null   float64
 13  nearest_hospital           6762 non-null   float

In [27]:
pois.to_csv(poi_csv)
townships.to_csv(township_csv)
transactions.to_csv(transaction_csv)

In [28]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217814 entries, 0 to 217938
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   project_id        217814 non-null  int64  
 1   project_name      217814 non-null  object 
 2   transacted_price  217814 non-null  int64  
 3   unit_price_psf    217781 non-null  float64
 4   date              217814 non-null  int64  
 5   property_type     217814 non-null  object 
 6   tenure            217814 non-null  object 
 7   floor             85842 non-null   object 
 8   area_sqft         217814 non-null  float64
 9   non_landed        217814 non-null  int64  
 10  bedrooms          217814 non-null  float64
 11  psf               217814 non-null  float64
 12  price             217814 non-null  int64  
 13  state             217814 non-null  object 
 14  planning_region   217814 non-null  object 
dtypes: float64(4), int64(5), object(6)
memory usage: 26.6+ MB
